# Forecasting future e-commerce sales

In the dynamic landscape of e-commerce, accurately predicting units sold is crucial for ensuring optimal inventory management and maximizing revenue. Moreover, understanding the impact of marketing expenditure, especially during campaign periods, is essential for crafting effective strategies to drive sales and customer engagement. 

By comparing forecasting models like VARIMA from DARTS library, Prophet by Meta, and ForecasterAutoreg from SKForecast, e-commerce businesses can unlock powerful insights into consumer behavior and market trends. These models offer sophisticated tools to analyze historical sales data alongside marketing initiatives, enabling businesses to forecast units sold with greater precision and anticipate the effects of marketing campaigns or web traffic trends on future sales. Through this comparative analysis, e-commerce stakeholders can enhance their predictive capabilities, optimize resource allocation, and make data-driven decisions that propel their businesses to success in the competitive online marketplace.

## Pre-processing our data

In [ ]:
# Requirements needed
!pip install -r requirements.txt

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Error metrics to measure our models performance
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Loading the data
data = pd.read_csv('time_series_dataframe.csv')

print("Are there any missing values?")
data.isna().any()

### Fill the missing values
To handle a specific column and interpolate missing values when seven consecutive days are missing, you can use the interpolate()

method='linear': Assumes a linear relationship between consecutive points, regardless of the time intervals between them. It works well when the time intervals between data points are relatively constant.

method='time': Takes into account the time intervals between data points when interpolating. It considers the time component, making it more suitable for time series data where the intervals between data points are irregular or unevenly spaced.

In [ ]:
# Value imputation using linear interpolation
data = data.interpolate(method='linear')

print("Are there any missing values?")
data.isna().any()

In [ ]:
# Convert columns to the right types
data['date'] = pd.to_datetime(data['date'])
data['users'] = pd.to_numeric(data['users'], errors='coerce')
data['sessions'] = pd.to_numeric(data['sessions'], errors='coerce')
data['marketing_cost'] = pd.to_numeric(data['marketing_cost'], errors='coerce')
data['clicks'] = pd.to_numeric(data['clicks'], errors='coerce')
data['impressions'] = pd.to_numeric(data['impressions'], errors='coerce')
data['click_through_rate'] = pd.to_numeric(data['click_through_rate'], errors='coerce')
data['avg_time_on_page'] = pd.to_numeric(data['avg_time_on_page'], errors='coerce')
data['bouce_rate'] = pd.to_numeric(data['bouce_rate'], errors='coerce')
data['units_sold'] = pd.to_numeric(data['units_sold'], errors='coerce')
data['is_campaign_period'] = pd.to_numeric(data['is_campaign_period'], errors='coerce')
data['day_of_week'] = pd.to_numeric(data['day_of_week'], errors='coerce')
data['month_number'] = pd.to_numeric(data['month_number'], errors='coerce')

### Define the parameters for the models
We need to define what are the amount of periods (days in this case) we want to predict and the last date of our training set. In other words, we define the date where the predictions will start.


In [ ]:
periods_to_predict = 30
end_training_date = '2023-02-19'
# end_training_date = '2023-03-19'

### Split our data into train and test
Based on the defined parameters, the data gets split for training and forecasting

In [ ]:
# Filter data before the determined end training date
data_train = data[data['date'] < end_training_date]

# Create date with end_training_date + periods_to_predict
last_forecasting_date = data_train.date.max() + pd.Timedelta(days=periods_to_predict + 1)

# Create dataframe with all data (past and future). This is important because we need to feed the 
#   models with future regressors
all_data = data[data['date'] < last_forecasting_date]

# Create dataframe with all available data until end training date
data_test = data[(data['date'] < last_forecasting_date) & (data['date'] >= end_training_date)]

# 




## DARTS: Forecasting with VARIMA

In [ ]:
# Import libraries
from darts.models import VARIMA
from darts import TimeSeries

In [ ]:
# Selecting relevant columns for analysis
data_train_selected = data_train[['date', 'users', 'units_sold', 'sessions', 
                                  'clicks', 'impressions', 'click_through_rate', 'avg_time_on_page']]

# Converting dataframe to a time series
series = TimeSeries.from_dataframe(data_train_selected, 'date', ['units_sold', 'users', 'sessions', 
                                                                 'avg_time_on_page', 'click_through_rate'])

# Since we want to be more accurate on our forecasts and explain what impacts our predictions, 
#   we include regressors or exogenous variables in the models
future_cov = TimeSeries.from_dataframe(all_data, 'date', ['marketing_cost', 'is_campaign_period'])
pred_cov = TimeSeries.from_dataframe(data_test, 'date', ['marketing_cost', 'is_campaign_period'])

# Initializing the VARIMA model with trend
model = VARIMA(trend = 'ct')

# Fitting the VARIMA model
model.fit(series, future_covariates = future_cov)

# Generating predictions for the next 15 time points
pred = model.predict(periods_to_predict, future_covariates = pred_cov)
pred.values()

# Extracting forecasted values for 'units_sold'
forecast_values = pred.univariate_component('units_sold').values()

# Extracting actual values for 'units_sold'
actual_data = data_test.tail(periods_to_predict)['units_sold'].values

# Calculating metrics to evaluate model performance
mae_darts = mean_absolute_error(actual_data, forecast_values)
rmse_darts = np.sqrt(mean_squared_error(actual_data, forecast_values))

# Print results
print("mean_absolute_error:", mae_darts)
print("mean_squared_error:", rmse_darts)

# Extracting dates for x-axis
dates = pd.to_datetime(data_test.tail(periods_to_predict)['date'])

# Plotting actual values
plt.plot(dates, actual_data, label='Actual')

# Plotting forecasted values
plt.plot(dates, forecast_values, label='Forecast', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Units Sold')
plt.title('Actual vs Forecasted Values')
plt.legend()
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent overlapping labels
plt.show()


# 




## Machine Learning approach: SKForecast

In [ ]:
# Import libraries
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def process_data_skforecast(df):
    """
    Process the DataFrame by converting the 'date' column to datetime format,
    setting it as the index, resampling to daily frequency, and sorting by date.
    
    Args:
    df (DataFrame): Input DataFrame with a 'date' column
    
    Returns:
    DataFrame: Processed DataFrame with datetime index
    """
    # Convert 'date' column to datetime format
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    
    # Set 'date' column as index
    df = df.set_index('date')
    
    # Resample to daily frequency
    df = df.asfreq('D')
    
    # Sort DataFrame by date
    df = df.sort_index()
    
    return df

# Apply function over datasets for skforecast
data_train_skforecast = process_data_skforecast(data_train)
data_test_skforecast = process_data_skforecast(data_test)

### Autoregressive model

A autoregressive model (ForecasterAutoreg) is trained using a linear regressor with Ridge regularization, and a time window of 2 weeks (14 lags). The latter means that, for each prediction, the traffic the website had in the previous 14 days is used as predictors.

Ridge models require predictors to be standardized. A StandardScaler is added to the forecaster using the argument transformer_y.

In [ ]:
# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 14
             )

forecaster.fit(
    y    = data_train_skforecast['units_sold'],
    exog = data_train_skforecast[['marketing_cost', 'is_campaign_period']]
)

forecaster

In [ ]:
# Predict
# ==============================================================================
steps = periods_to_predict
predictions = forecaster.predict(
                  steps = steps,
                  exog = data_test_skforecast[['marketing_cost', 'is_campaign_period']]
              )

In [ ]:
# Calculating metrics to evaluate model performance
mae_skforecast = mean_absolute_error(actual_data, predictions)
rmse_skforecast = mean_squared_error(actual_data, predictions, squared=False)

# Print the results
print("mean_absolute_error:", mae_skforecast)
print("mean_squared_error:", rmse_skforecast)

# Plotting actual values
plt.plot(data_test_skforecast.index, data_test['units_sold'], label='Actual')

# Plotting forecasted values
plt.plot(data_test_skforecast.index, predictions, label='Forecast', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Units Sold')
plt.title('Actual vs Forecasted Values')
plt.legend()
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent overlapping labels
plt.show()

In [ ]:
# Show what are the most important features that explain the predictions
#  Note: lag_1 refers to the values from the previous day; lag_2 to the values from 2 days ago; and so on
forecaster.get_feature_importances().sort_values(by='importance', ascending=False)

# 




## Forecasting with Prophet from Meta

In [ ]:
from prophet import Prophet

# Selecting relevant columns for analysis
data_train_selected = data_train[['date', 'users', 'units_sold', 'sessions', 'clicks', 'impressions', 'click_through_rate', 'avg_time_on_page', 'marketing_cost', 'is_campaign_period']]

# Converting dataframe to a time series
data_train_selected.columns = ['ds'] + data_train_selected.columns[1:].tolist()
data_train_selected = data_train_selected.rename(columns={'units_sold': 'y'})
series = data_train_selected

# Initializing the Prophet model
model = Prophet()

# Adding regressors to the model
for regressor in ['marketing_cost', 'is_campaign_period']:
    model.add_regressor(regressor)

# Fitting the Prophet model
model.fit(series)

# Creating future dataframe for prediction
future_df = model.make_future_dataframe(periods=periods_to_predict)

# Adding regressor values to the future dataframe
for regressor in ['marketing_cost', 'is_campaign_period']:
    future_df[regressor] = all_data[regressor].values

# Generating predictions
forecast = model.predict(future_df)

# Extracting forecasted values for 'units_sold'
forecast_values_prophet = forecast[['ds', 'yhat']].tail(periods_to_predict)['yhat'].values

# Extracting actual values
actual_data = data_test.tail(periods_to_predict)['units_sold'].values

# Calculating metrics to evaluate model performance
mae_prophet = mean_absolute_error(actual_data, forecast_values_prophet)
rmse_prophet = np.sqrt(mean_squared_error(actual_data, forecast_values_prophet))

print("mean_absolute_error:", mae_prophet)
print("mean_squared_error:", rmse_prophet)

# Plotting
plt.plot(data_test.tail(periods_to_predict)['date'], actual_data, label='Actual')
plt.plot(data_test.tail(periods_to_predict)['date'], forecast_values_prophet, label='Forecast', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Units Sold')
plt.title('Actual vs Forecasted Values')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# 




## Compare the 3 models

In [ ]:
# Create dataset with the required error values
model_error_comparison = {
    'MAE': {
        'prophet': mae_prophet,
        'skforecast': mae_skforecast,
        'darts': mae_darts
    },
    'RMSE': {
        'prophet': rmse_prophet,
        'skforecast': rmse_skforecast,
        'darts': rmse_darts
    }
}

# Create a DataFrame object and display it
model_error_comparison = pd.DataFrame(model_error_comparison).sort_values(by='MAE', ascending=True)
model_error_comparison

In [ ]:
# Extracting dates for x-axis
dates = pd.to_datetime(data_test.tail(periods_to_predict)['date'])

# Plotting actual values
plt.plot(dates, actual_data, label='Actual')

# Plotting forecasted values for each model
plt.plot(dates, forecast_values, label='DARTS', linestyle='dashed')
plt.plot(dates, predictions, label='SKForecast', linestyle='dashed')
plt.plot(dates, forecast_values_prophet, label='Prophet', linestyle='dashed')

# Show chart with all forecasts
plt.xlabel('Date')
plt.ylabel('Units Sold')
plt.title('Actual vs Forecasted Values')
plt.legend()
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent overlapping labels
plt.show()